In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import os
import numpy as np
sys.path.append(os.path.abspath(".."))
from preprocess import process_MyDailyTravelData

survey_path = Path("../configs/Chicago/data/person.csv")
synth_path = Path("../run/acer_runs/Chicago_20250518_1525/20250518_1525_results.csv")
config_folder = Path("../configs/Chicago")

survey_df = pd.read_csv(survey_path, low_memory=False)
synth_df = pd.read_csv(synth_path, index_col=0)
query_dict = process_MyDailyTravelData(config_folder=config_folder)